# Deploying a model + custom transformer on Managed Online Endpoint (MOE)

In [1]:
subscriptionID = '0cfe2870-d256-4119-b0a3-16293ac11bdc'
RG = '1-2ba149fb-playground-sandbox'
ws_name = "MLOPS101"

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

ws = MLClient(DefaultAzureCredential(), subscription_id = subscriptionID,
              resource_group_name = RG, workspace_name = ws_name)

print(ws)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fb715dea5c0>,
         subscription_id=0cfe2870-d256-4119-b0a3-16293ac11bdc,
         resource_group_name=1-2ba149fb-playground-sandbox,
         workspace_name=MLOPS101)


In [3]:
!wget https://raw.githubusercontent.com/Azure/azureml-examples/main/sdk/python/endpoints/batch/deploy-pipelines/training-with-components/data/train/heart.csv -P assets/data/

--2023-06-16 07:56:41--  https://raw.githubusercontent.com/Azure/azureml-examples/main/sdk/python/endpoints/batch/deploy-pipelines/training-with-components/data/train/heart.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13273 (13K) [text/plain]
Saving to: ‘assets/data/heart.csv’

heart.csv           100%[===================>]  12.96K  --.-KB/s    in 0.01s   

2023-06-16 07:56:41 (1.22 MB/s) - ‘assets/data/heart.csv’ saved [13273/13273]



In [4]:
from pathlib import Path

ROOT_DIR = Path('./assets')
ENV_DIR = ROOT_DIR / 'env'
TRAIN_DIR = ROOT_DIR / 'train'
DATA_DIR = ROOT_DIR / 'data'
DEPLOYMENT_DIR = ROOT_DIR / 'deployment'

ENV_DIR.mkdir(parents = True, exist_ok = True)
TRAIN_DIR.mkdir(parents = True, exist_ok = True)
DATA_DIR.mkdir(parents = True, exist_ok = True)
DEPLOYMENT_DIR.mkdir(parents = True, exist_ok = True)

In [5]:
from pathlib import Path
import pandas as pd
import numpy as np
import sklearn
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

transform_filename = 'column_transformer.pkl'
continuous_features = ['age', 'chol', 'oldpeak', 'thalach', 'trestbps']
discrete_features = ['ca', 'cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']
target_column = 'target'

def preprocessing_pipeline(categorical_encoding, cf, df): #cf -> Continuous feat df -> discrete feat
    try:
        if categorical_encoding == 'ordinal':
            cat_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
        elif categorical_encoding == 'onehot':
            cat_enc = OneHotEncoder(handle_unknown="ignore")
        else:
            raise NotImplementedError('Possible values are ordinal or onehot')

        conti_feat_pipeline = sklearn.pipeline.Pipeline([
            ('imputer', SimpleImputer(strategy = 'median')),
            ('scaler', StandardScaler())
        ])

        disc_feat_pipeline = sklearn.pipeline.Pipeline([

            ('imputer', SimpleImputer(strategy = 'most_frequent')),
            ('encoder', cat_enc)

        ])

        transformations = ColumnTransformer([
            ('conti_feat_pipeline', conti_feat_pipeline, cf),
            ('disc_feat_pipeline', disc_feat_pipeline, df)
        ])
        return transformations
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno, e)

def preprocess_data(dataframe, label, cf, df, cat_enc = 'ordinal', transformations = None):
    try:
    
        if label in dataframe.columns:
            X = dataframe.iloc[:,:-1]
            restore_target = True
        else:
            X = dataframe
            restore_target = False

        if transformations:
            X_transformed = transformations.transform(X)
        else:
            transformations = preprocessing_pipeline(cat_enc, cf, df)
            X_transformed = transformations.fit_transform(X)
        transformed_discrete_features = (
            transformations.transformers_[1][1]
            .named_steps["encoder"]
            .get_feature_names_out(discrete_features)
        )
        all_features = continuous_features + list(transformed_discrete_features)

        if restore_target:
            target_values = dataframe[label].to_numpy().reshape(len(dataframe), 1)
            X_transformed = np.hstack((X_transformed, target_values))
            all_features.append(label)

        return pd.DataFrame(X_transformed, columns = all_features), transformations
    
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno, e)

temp = pd.read_csv('./assets/data/heart.csv')
preprocessed, transformations = preprocess_data(
    temp,
    target_column,
    continuous_features,
    discrete_features,
    'ordinal'
)
preprocessed.to_csv('./assets/data/transformed_heart.csv', index=False)
joblib.dump(transformations, './assets/train/col_transformer.pkl')

['./assets/train/col_transformer.pkl']

In [6]:
from distutils.util import strtobool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from xgboost import XGBClassifier

df = pd.read_csv('./assets/data/transformed_heart.csv')

train, test = train_test_split(df, test_size = 0.2)
train_features = train.drop(columns=[target_column])
train_target = train[target_column]

model = XGBClassifier(scale_pos_weight=99)
model.fit(train_features, train_target)

test_features = test.drop(columns=[target_column])
predictions = model.predict(test_features)

test['Labels'] = predictions
test['Probabilities'] = model.predict_proba(test_features)[:, 1]

test.to_csv('./assets/data/transformed_heart_test.csv', index=False)

accuracy = accuracy_score(test[target_column], predictions)
recall = recall_score(test[target_column], predictions)

print({'accuracy': accuracy, 'recall': recall})

{'accuracy': 0.7868852459016393, 'recall': 0.625}


In [7]:
import joblib

joblib.dump(model, './assets/train/xgb.pkl') 

['./assets/train/xgb.pkl']

In [8]:
from sklearn.pipeline import Pipeline

transformer = joblib.load('./assets/train/col_transformer.pkl')
xgbc = joblib.load('./assets/train/xgb.pkl')

pipeline = Pipeline(steps=[('transformer', transformer),
                              ('xgbc', xgbc)
                    ])

In [9]:
pipeline.fit(train_features, train_target)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('conti_feat_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'chol', 'oldpeak',
                                                   'thalach', 'trestbps']),
                                                 ('disc_feat_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(handle_unk...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [10]:
pipeline.predict(test_features)

array([1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0])

train.dtypes

# ONNX 101

<pre>

<b>ONNX - Open Neural Network eXchange Format</b>

1. Interoperability between many formats like PyTorch, TF and Sklearn
2. This gives a variety of functions that converts framework agnostic capabilities
3. It is run in an ONNX runtime and on top of any interpreter like Java, Python and etc.
4. ONNX produces a computational graph that consists of how the model should be executed
5. ONNX operators are for inputs, outputs, Node, initializer and attributes
6. Nodes are matrix multiplication, add etc.
7. Initializers can be default values
8. Attributes are frozen values in an operator like alpha, beta
9. Serialization is done through protobuf
10. There are 2 domains ai.onnx and ai.onnx.ml. Domains are set of operators available for us. ai.onnx has DL modules while ai.onnx.ml has sklearn related implementations. Custom domains are also possible.
11. Tensors in ONNX have a type, shape and are contiguous in memory.
12. Element types are data types
13. Opset is the version of ONNX package. Every new opset will introduce a new set of operators.
14. Converters are used to convert the models into ONNX format.
    a. sklearn-onnx: converts models from scikit-learn,
    b. tensorflow-onnx: converts models from tensorflow,
    c. onnxmltools: converts models from lightgbm, xgboost, pyspark, libsvm
    d. torch.onnx: converts model from pytorch.

</pre>

# ONNX Sklearn Definition

<pre>

1. Define the input types:
    Eg. [
            ('float_input', FloatTensorType([None, 4])), # We are saying that 4 inputs are float
            ('int_input', FloatTensorType([None, 2])) # We are saying that 2 inputs are int
    ]
2. Convert the model to ONNX format by passing the model and input types
3. Using inference session predict the results
4. Skl2onnx only supports scikit-learn entities when we have xgboost, lightgbm along with sklearn in the pipeline we have to use onnxmltools to register and use it. If we do not do that it will result in error.
</pre>

Ref : https://onnx.ai/sklearn-onnx/auto_examples/plot_pipeline_xgboost.html

In [11]:
import onnxruntime
from skl2onnx.helpers import collect_intermediate_steps, compare_objects
from skl2onnx.common.data_types import FloatTensorType

operators = collect_intermediate_steps(pipeline, "pipeline", [("input",
                                         FloatTensorType([None, len(train_features.columns)]))])
dummy_data = train.iloc[:2,:].to_numpy()

for op in operators:

    # The ONNX for this operator.
    onnx_step = op['onnx_step']

    # Use onnxruntime to compute ONNX outputs
    sess = onnxruntime.InferenceSession(onnx_step.SerializeToString(),
                                        providers=["CPUExecutionProvider"])

    # Let's use the initial data as the ONNX model
    # contains all nodes from the first inputs to this node.
    onnx_outputs = sess.run(None, {'input': data})
    onnx_output = onnx_outputs[0]
    skl_outputs = op['model']._debug.outputs['transform']

    # Compares the outputs between scikit-learn and onnxruntime.
    assert_almost_equal(onnx_output, skl_outputs)

    # A function which is able to deal with different types.
    compare_objects(onnx_output, skl_outputs)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/skl2onnx/algebra/onnx_ops.py:159: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  tys = obj.typeStr or ''
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/skl2onnx/algebra/automation.py:154: UserWarning: OpSchema.FormalParameter.isHomogeneous is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.is_homogeneous instead.
  if getattr(obj, 'isHomogeneous', False):
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/jinja2/environment.py:485: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  return getattr(obj, attribute)


RuntimeError: Unable to find column name 'age' among names ['input']. Make sure the input names specified with parameter initial_types fits the column names specified in the pipeline to convert. This may happen because a ColumnTransformer follows a transformer without any mapped converter in a pipeline.

In [12]:
import skl2onnx
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  # noqa
import onnxmltools.convert.common.data_types

skl2onnx.update_registered_converter(XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

In [13]:
def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([None, 1])
        elif v == 'float64':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs

input_types = convert_dataframe_schema(train_features)

In [14]:
model_onnx = skl2onnx.convert_sklearn(
    pipeline, 'pipeline_heart_classification',
    input_types,
    target_opset = {'': 12, 'ai.onnx.ml': 2})

# And save.
with open(f'{TRAIN_DIR}/pipeline_xgboost.onnx', 'wb') as f:
    f.write(model_onnx.SerializeToString())

In [15]:
input_types

[('age', FloatTensorType(shape=[None, 1])),
 ('chol', FloatTensorType(shape=[None, 1])),
 ('oldpeak', FloatTensorType(shape=[None, 1])),
 ('thalach', FloatTensorType(shape=[None, 1])),
 ('trestbps', FloatTensorType(shape=[None, 1])),
 ('ca', FloatTensorType(shape=[None, 1])),
 ('cp', FloatTensorType(shape=[None, 1])),
 ('exang', FloatTensorType(shape=[None, 1])),
 ('fbs', FloatTensorType(shape=[None, 1])),
 ('restecg', FloatTensorType(shape=[None, 1])),
 ('sex', FloatTensorType(shape=[None, 1])),
 ('slope', FloatTensorType(shape=[None, 1])),
 ('thal', FloatTensorType(shape=[None, 1]))]

In [16]:
import onnxruntime as rt

sess = rt.InferenceSession(f'{TRAIN_DIR}/pipeline_xgboost.onnx',
                           providers=["CPUExecutionProvider"])

In [17]:
dir(sess)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_inference_session',
 '_enable_fallback',
 '_fallback_providers',
 '_inputs_meta',
 '_model_bytes',
 '_model_meta',
 '_model_path',
 '_outputs_meta',
 '_overridable_initializers',
 '_profiling_start_time_ns',
 '_provider_options',
 '_providers',
 '_read_config_from_model',
 '_reset_session',
 '_sess',
 '_sess_options',
 '_sess_options_initial',
 '_validate_input',
 'disable_fallback',
 'enable_fallback',
 'end_profiling',
 'get_inputs',
 'get_modelmeta',
 'get_outputs',
 'get_overridable_initializers',
 'get_profiling_start_time_ns',
 'get_provider_options',
 'get_providers',
 'get_session_options',
 'get_tuning_r

In [18]:
sess.get_inputs()

In [19]:
dir(sess.get_inputs()[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'name',
 'shape',
 'type']

In [20]:
sess.get_outputs()

In [21]:
dir(sess.get_outputs()[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'name',
 'shape',
 'type']

In [22]:
inputs = {c: test_features[c].values for c in test_features.columns}
for c in test_features.columns:
    inputs[c] = inputs[c].astype(np.float32)
for k in inputs:
    inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))

In [23]:
pred_onx = sess.run(None, inputs)

In [24]:
inputs['age'] #This is how the input looks like

array([[ 0.82274705],
       [-0.28818145],
       [ 0.4894685 ],
       [ 0.15618995],
       [-1.9545742 ],
       [-0.1770886 ],
       [ 1.2671185 ],
       [ 0.15618995],
       [ 0.9338399 ],
       [ 0.37837565],
       [ 0.9338399 ],
       [-2.843317  ],
       [ 0.0450971 ],
       [ 0.37837565],
       [ 0.82274705],
       [-1.5102028 ],
       [-1.5102028 ],
       [-1.0658314 ],
       [ 0.4894685 ],
       [-1.0658314 ],
       [ 0.4894685 ],
       [ 1.4893042 ],
       [-1.5102028 ],
       [-0.7325528 ],
       [ 0.4894685 ],
       [-1.39911   ],
       [-1.1769242 ],
       [-0.3992743 ],
       [-1.1769242 ],
       [ 0.7116542 ],
       [ 0.26728278],
       [ 0.9338399 ],
       [-1.1769242 ],
       [-0.06599575],
       [-0.62146   ],
       [-1.6212957 ],
       [-1.5102028 ],
       [-0.7325528 ],
       [-1.39911   ],
       [ 0.0450971 ],
       [ 0.15618995],
       [-0.3992743 ],
       [-1.1769242 ],
       [-0.7325528 ],
       [ 0.37837565],
       [ 1

In [25]:
dir(pred_onx)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [26]:
len(pred_onx)

2

In [27]:
len(pred_onx[0])

61

In [28]:
pred_onx[0][:5]

array([1, 1, 1, 0, 0], dtype=int64)

## Creating an online endpoint

In [29]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

In [30]:
endpoint_name = 'heart-onnx-16-6'

endpoint = ManagedOnlineEndpoint(
    name = endpoint_name,
    description = 'this is a sample online endpoint',
    auth_mode = 'key',
    tags = {'env': 'test'},
)

<pre>

<b> The following things are required to host the model on AML Endpoint </b>

> Model files - here it is an ONNX file
> Scoring script - how do we want to predict/evaluate the incoming data. Score.py is the default filename
> Environment - A custom environment which runs on docker
> Settings - Related to compute etc.
> EndpointName - The name of the endpoint where the pipeline is hosted

</pre>

In [31]:
%%writefile {ENV_DIR}/conda_definition.yml

name: heart-env
channels:
- conda-forge
dependencies:
- python=3.10
- pip
- pip:
  - pandas==2.0.2
  - numpy==1.24.3
  - onnxruntime==1.15.0
  - inference-schema==1.5.1
  - azureml-inference-server-http==0.8.4

Writing assets/env/conda_definition.yml


In [33]:
model = Model(path = './assets/train/pipeline_xgboost.onnx')
env = Environment(
    conda_file="./assets/env/conda_definition.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [34]:
endpoint = ws.online_endpoints.begin_create_or_update(endpoint).result()

In [35]:
key = ws.online_endpoints.get_keys(endpoint_name).primary_key

In [50]:
%%writefile {DEPLOYMENT_DIR}/score.py

#Writing inference schema to generate Swagger documentation automatically

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.standard_py_parameter_type import (
    StandardPythonParameterType,
)
import os
import numpy as np
import logging
import onnxruntime as rt

model = None
targets = np.array(['Yes', 'No'])

def init():
    global model
    global ip_features
    model_dir = os.getenv('AZUREML_MODEL_DIR', '')
    model_path = os.path.join(model_dir, 'pipeline_xgboost.onnx')
    model = rt.InferenceSession(model_path,
                           providers=["CPUExecutionProvider"])
    ip_features = ['age', 'chol', 'oldpeak', 'thalach', 'trestbps', 'ca', 'cp', 'exang',
       'fbs', 'restecg', 'sex', 'slope', 'thal']

@input_schema(
    param_name = 'data', param_type = NumpyParameterType(np.array([[ 0.15618995, 0.79578295, 2.52965595, -0.70000655,  3.84978957,2., 4.,  1.,  1.,  2.,0.,  2.,  4.]]))
)
@output_schema(output_type=StandardPythonParameterType({'Disease': ['Yes']}))
def run(data):
    
    logging.info(type(data))
    
    inputs = {column : [] for column in ip_features}
    for row in data:
        for index, col in enumerate(ip_features):
            inputs[col].append(row[index])
    
    for c in ip_features:
        inputs[c] = np.array(inputs[c]).astype(np.float32)
        
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))
    pred_onx = model.run(None, inputs)
    predicted_categories = np.choose(pred_onx[0], targets).flatten()
    result = {
        "Disease": predicted_categories.tolist(),
    }
    return result

Overwriting assets/deployment/score.py


In [51]:
blue_deployment = ManagedOnlineDeployment(
    name = 'blue',
    endpoint_name = endpoint_name,
    model = model,
    environment = env,
    code_configuration = CodeConfiguration(
        code = './assets/deployment', scoring_script = 'score.py'
    ),
    instance_type='Standard_DS1_v2',
    instance_count=1,
)

In [52]:
deployment = ws.online_deployments.begin_create_or_update(blue_deployment).result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint heart-onnx-16-6 exists
Uploading deployment (0.0 MBs): 100%|██████████| 1743/1743 [00:00<00:00, 106732.93it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...................................

In [53]:
blue_deployment = ws.online_deployments.get(
    name="blue", endpoint_name = endpoint_name
)
##--Scaling the deployment--##
#blue_deployment.instance_count = 2
#ws.online_deployments.begin_create_or_update(blue_deployment).result()

In [54]:
print(endpoint.traffic)
print(endpoint.scoring_uri)

{}
https://heart-onnx-16-6.eastus2.inference.ml.azure.com/score


In [55]:
#Green deployments should be different but in our case we are taking the same code

green_deployment = ManagedOnlineDeployment(
    name = 'green',
    endpoint_name = endpoint_name,
    model = model,
    environment = env,
    code_configuration = CodeConfiguration(
        code = './assets/deployment', scoring_script = 'score.py'
    ),
    instance_type='Standard_DS1_v2',
    instance_count=1,
)

In [56]:
ws.online_deployments.begin_create_or_update(green_deployment).result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint heart-onnx-16-6 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


.......................................................................

ManagedOnlineDeployment({'private_network_connection': False, 'provisioning_state': 'Succeeded', 'data_collector': None, 'endpoint_name': 'heart-onnx-16-6', 'type': 'Managed', 'name': 'green', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/0cfe2870-d256-4119-b0a3-16293ac11bdc/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/od:b1b80a69-b296-4608-92a4-33acdff57c2e:13b04c38-768d-4e71-b095-68eb12a2ace1?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/0cfe2870-d256-4119-b0a3-16293ac11bdc/resourceGroups/1-2ba149fb-playground-sandbox/providers/Microsoft.MachineLearningServices/workspaces/MLOPS101/onlineEndpoints/heart-onnx-16-6/deployments/green', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlops101x/code/Users/cloud_user_p_b28104ff', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object

In [57]:
%%writefile test.json

{
  "data": [
    [
      0.15618995,
      0.79578295,
      2.52965595,
      -0.70000655,
      3.84978957,
      2,
      4,
      1,
      1,
      2,
      0,
      2,
      4
    ]
  ]
}

Writing test.json


In [58]:
ws.online_endpoints.invoke(
    endpoint_name = endpoint_name,
    deployment_name = 'green',
    request_file = 'test.json'
)

'{"Disease": ["No"]}'

In [ ]:
endpoint.traffic = {'blue': 90, 'green' : 10}
ws.online_endpoints.begin_create_or_update(endpoint).result()

In [62]:
ws.online_deployments.begin_delete(
    name = 'blue', endpoint_name = endpoint_name
).wait()